In [1]:
# Script to generate variations on David's 2-BTP model to run methane flame speeds to compare with NIST

In [47]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
import numpy as np
import subprocess


import copy
import os

In [3]:
# load David's 2-BTP model, minus the halogens
# https://github.com/comocheng/halogen_models/tree/main/combustion_symposium_paper/supporting_material

RMG_chemkin_path = 'models/RMG_no_halogens/no_halogens.inp'
RMG_dictionary_path = 'models/RMG_no_halogens/no_halogens_dict.txt'
RMG_transport_path = 'models/RMG_no_halogens/no_halogens_tran.dat'
RMG_cti_path = 'models/RMG_no_halogens/no_halogens.cti'

RMG_species_list, RMG_reaction_list = rmgpy.chemkin.load_chemkin_file(RMG_chemkin_path, dictionary_path=RMG_dictionary_path, transport_path=RMG_transport_path)
RMG_gas = ct.Solution(RMG_cti_path)



In [6]:
# Load NIST 2-BTP model
# /scratch/westgroup/David/halogen_models/models/2-BTP/NIST/2-BTP_kinetics.cti
NIST_cti_path = 'models/NIST/2-BTP_kinetics.cti'
NIST_dictionary_path = 'models/NIST/species_dictionary.txt'
NIST_chemkin_path = 'models/NIST/pym_gas_copy.inp'

NIST_species_list, NIST_reaction_list = rmgpy.chemkin.load_chemkin_file(NIST_chemkin_path, dictionary_path=NIST_dictionary_path)
NIST_gas = ct.Solution(NIST_cti_path)

For species CH2OCH, discontinuity in cp/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  8.393471510000001
	Value computed using high-temperature polynomial: 9.1801039121875

For species CH2OCH, discontinuity in h/RT detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  42.199147089791666
	Value computed using high-temperature polynomial: 41.961461604875005

For species CH2OCH, discontinuity in s/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  33.70692865946735
	Value computed using high-temperature polynomial: 33.51209988778391

For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  47.65235236593109
	Value computed using high-temperature polynomial: 48.43623165666667

For species C4H5-2, discontinuity in s/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  52.42918829260522
	Value computed using high-temperature p

In [11]:
# Species Diff
RMG2NIST_mapping = {}
NIST2RMG_mapping = {}
for i, rmg_sp in enumerate(RMG_species_list):
    for j, nist_sp in enumerate(NIST_species_list):
        if rmg_sp.is_isomorphic(nist_sp):
            RMG2NIST_mapping[i] = j
            NIST2RMG_mapping[j] = i

# Reaction Diff
RMG2NIST_rxn_mapping = {}
NIST2RMG__rxn_mapping = {}
for i, rmg_sp in enumerate(RMG_reaction_list):
    for j, nist_sp in enumerate(NIST_reaction_list):
        if rmg_sp.is_isomorphic(nist_sp):
            RMG2NIST_rxn_mapping[i] = j
            NIST2RMG__rxn_mapping[j] = i

# count the common reactions:
print(len(RMG2NIST_rxn_mapping))

240


In [40]:
RMG2NIST_rxn_mapping

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 7: 10,
 10: 9,
 11: 8,
 13: 11,
 14: 12,
 15: 15,
 16: 14,
 17: 16,
 18: 18,
 19: 17,
 20: 13,
 21: 20,
 22: 19,
 23: 21,
 24: 22,
 25: 23,
 26: 25,
 27: 24,
 28: 26,
 29: 31,
 30: 27,
 31: 28,
 32: 29,
 33: 30,
 34: 33,
 38: 38,
 39: 39,
 40: 40,
 42: 41,
 43: 42,
 47: 43,
 48: 44,
 49: 46,
 51: 48,
 52: 49,
 53: 53,
 54: 50,
 56: 52,
 61: 56,
 64: 58,
 65: 59,
 67: 60,
 69: 63,
 70: 64,
 72: 67,
 73: 68,
 76: 69,
 77: 70,
 78: 71,
 79: 45,
 80: 34,
 81: 74,
 82: 75,
 83: 76,
 84: 77,
 85: 78,
 86: 79,
 91: 80,
 92: 81,
 94: 82,
 95: 83,
 96: 84,
 98: 87,
 99: 88,
 100: 85,
 101: 86,
 103: 91,
 104: 94,
 105: 95,
 106: 96,
 107: 97,
 108: 92,
 109: 93,
 110: 73,
 111: 100,
 112: 101,
 113: 102,
 114: 103,
 115: 104,
 116: 105,
 117: 106,
 118: 107,
 121: 72,
 122: 108,
 123: 109,
 124: 110,
 125: 111,
 126: 112,
 127: 113,
 128: 114,
 130: 115,
 131: 116,
 132: 117,
 133: 89,
 134: 118,
 135: 119,
 136: 120,
 137: 121,
 138: 122,
 139: 123,
 140: 12

In [66]:
# Convert the NIST species in the reactions to RMG species, but keep the NIST kinetics
def NIST2RMG(nist_reaction):
    rmg_reaction = copy.deepcopy(nist_reaction)
    reactants = []
    for reactant in nist_reaction.reactants:
        NIST_species_index = NIST_species_list.index(reactant)
        reactants.append(RMG_species_list[NIST2RMG_mapping[NIST_species_index]])
    rmg_reaction.reactants = reactants
    
    products = []
    for product in nist_reaction.products:
        NIST_species_index = NIST_species_list.index(product)
        products.append(RMG_species_list[NIST2RMG_mapping[NIST_species_index]])
    rmg_reaction.products = products
    
    return rmg_reaction


In [70]:
# Create a new model using all values from the NIST model that are common - change reactions only
RMG_NIST_path = os.path.join('models', 'RMG_NIST')
os.makedirs(RMG_NIST_path, exist_ok=True)

# RMG_NIST_species = []
RMG_NIST_species = RMG_species_list
RMG_NIST_reactions = []
# for i, species in enumerate(RMG_species_list):
#     if i in RMG2NIST_mapping.keys():
#         RMG_NIST_species.append(NIST_species_list[RMG2NIST_mapping[i]])
#     else:
#         RMG_NIST_species.append(species)
        
for i, rxn in enumerate(RMG_reaction_list):
    if i in RMG2NIST_rxn_mapping.keys():
        RMG_NIST_reactions.append(NIST2RMG(NIST_reaction_list[RMG2NIST_rxn_mapping[i]]))
    else:
        RMG_NIST_reactions.append(rxn)


        
chemkin_file = os.path.join(RMG_NIST_path, 'RMG_NIST.inp')
transport_file = os.path.join(RMG_NIST_path, 'RMG_NIST_tran.dat')
        

rmgpy.chemkin.save_chemkin_file(chemkin_file, RMG_NIST_species, RMG_NIST_reactions, verbose=True, check_for_duplicates=True)
rmgpy.chemkin.save_transport_file(transport_file, RMG_NIST_species)
rmgpy.chemkin.save_species_dictionary(os.path.join(RMG_NIST_path, 'RMG_NIST_dict.txt'), RMG_NIST_species)

In [71]:
subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport_file}'])

# cantera.ck2cti.convertMech(os.path.join(RMG_NIST_path, 'RMG_NIST.inp'), transport=os.path.join(RMG_NIST_path, 'RMG_NIST_tran.dat'))

Wrote CTI mechanism file to 'models/RMG_NIST/RMG_NIST.cti'.
Mechanism contains 98 species and 833 reactions.
Validating mechanism...CanteraWarning: Troe::init: Unexpected parameter value T2=0. Omitting exp(T2/T) term from falloff expression. To suppress this warning, remove value for T2 from the input file. In the unlikely case that the exp(T2/T) term should be included with T2 effectively equal to 0, set T2 to a sufficiently small value (i.e. T2 < 1e-16).
PASSED.


CompletedProcess(args=['ck2cti', '--input=models/RMG_NIST/RMG_NIST.inp', '--transport=models/RMG_NIST/RMG_NIST_tran.dat'], returncode=0)

In [73]:
common_species

[Species(label="N2", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[3.53101,-0.000123661,-5.02999e-07,2.43531e-09,-1.40881e-12,-1046.98,2.96747], Tmin=(200,'K'), Tmax=(1000,'K')), NASAPolynomial(coeffs=[2.95258,0.0013969,-4.92632e-07,7.8601e-11,-4.60755e-15,-923.949,5.87189], Tmin=(1000,'K'), Tmax=(6000,'K'))], Tmin=(200,'K'), Tmax=(6000,'K'), comment="""Thermo library: primaryThermoLibrary"""), molecule=[Molecule(smiles="N#N")], transport_data=TransportData(shapeIndex=1, epsilon=(810.91,'J/mol'), sigma=(3.621,'angstrom'), dipoleMoment=(0,'De'), polarizability=(1.76,'angstrom^3'), rotrelaxcollnum=4.0, comment="""PrimaryTransportLibrary"""), molecular_weight=(28.0137,'amu')),
 Species(index=3, label="CH4", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[4.2054,-0.00535541,2.51118e-05,-2.13756e-08,5.97496e-12,-10161.9,-0.921227], Tmin=(100,'K'), Tmax=(1084.15,'K')), NASAPolynomial(coeffs=[0.908349,0.0114539,-4.57166e-06,8.29174e-10,-5.663e-14,-9720.01,13.9926], Tmin=(1084.15,'K'), Tmax=

In [74]:
# create a model that only has RMG-NIST common reactions
common_path = os.path.join('models', 'common')
os.makedirs(common_path, exist_ok=True)

common_species = []
common_reactions = []
for i in range(0, len(RMG_species_list)):
    if i in RMG2NIST_mapping.keys():
        common_species.append(RMG_species_list[i])

for i in range(0, len(RMG_reaction_list)):
    if i in RMG2NIST_rxn_mapping.keys():
        common_reactions.append(RMG_reaction_list[i])

    
chemkin_file = os.path.join(common_path, 'common.inp')
transport_file = os.path.join(common_path, 'common.dat')
        

rmgpy.chemkin.save_chemkin_file(chemkin_file, common_species, common_reactions, verbose=True, check_for_duplicates=True)
rmgpy.chemkin.save_transport_file(transport_file, common_species)
rmgpy.chemkin.save_species_dictionary(os.path.join(common_path, 'common_dict.txt'), common_species)

subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport_file}'])

Wrote CTI mechanism file to 'models/common/common.cti'.
Mechanism contains 45 species and 246 reactions.
Validating mechanism...PASSED.


CompletedProcess(args=['ck2cti', '--input=models/common/common.inp', '--transport=models/common/common.dat'], returncode=0)

In [24]:
dir(cantera.ck2cti)

['Arrhenius',
 'Avogadro',
 'Chebyshev',
 'ChemicallyActivated',
 'ENERGY_UNITS',
 'Falloff',
 'InputParseError',
 'KineticsData',
 'KineticsModel',
 'MultiNASA',
 'NASA',
 'PDepArrhenius',
 'Parser',
 'QUANTITY_UNITS',
 'Reaction',
 'Species',
 'Sri',
 'Surface',
 'SurfaceArrhenius',
 'ThermoModel',
 'ThirdBody',
 'TransportData',
 'Troe',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_open',
 'compatible_quantities',
 'contains',
 'convertMech',
 'defaultdict',
 'fortFloat',
 'get_index',
 'getopt',
 'itertools',
 'logging',
 'main',
 'np',
 'open',
 'os',
 'print_function',
 're',
 'script_entry_point',
 'string_types',
 'strip_nonascii',
 'sys']

In [5]:
len(RMG_reaction_list)

825